In [1]:

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)


filePath = './../../binaries/gnuit/src/gitfm'

# filePath = './../../binaries/c_many/stacktest'


In [2]:
def get_DIE_at_offset(CU, offset):
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None

def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_pointer_type':
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU,_attr.value) 
        
    
    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")
    
    


In [3]:
from collections import defaultdict
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()
    
    
    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()

In [4]:
def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)

In [5]:
# https://github.com/eliben/pyelftools/blob/master/examples/dwarf_location_info.py


from elftools.dwarf.locationlists import LocationParser, LocationExpr
FUNC_PARAMS = {}
def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()
        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        

        cc=0
        for CU in dwarfinfo.iter_CUs():
            if cc>0:
                continue
            cc+=1
            #get source file name
            print(CU)
            line_program = dwarfinfo.line_program_for_CU(CU)
            print('TEST!!!')
            filename_map = line_entry_mapping(line_program)
            CU_src_file =  list(filename_map.items())[0][0]
            
            CU_dictionary_key = CU_src_file 
            
            FUNC_PARAMS[CU_dictionary_key] = {}
            
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = None
            print('____'*10)
            for DIE in CU.iter_DIEs():
                
                ############################################################
                #############   Prasing Function DIEs start ################
                
                
                if DIE.tag == 'DW_TAG_subprogram':
                    are_DIEs_of_function = True
                    
                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name] ={}
                            print("SUBPROGRAM: ",FUNC_name)
                            
                if DIE.tag == 'DW_TAG_formal_parameter' or DIE.tag =='DW_TAG_variable':
                    tags = [attr.name for attr in DIE.attributes.values()]
                    PARAM_name = None
                    if FUNC_name==None:

                        FUNC_name ="global"
                        
                        if FUNC_name not in FUNC_PARAMS[CU_dictionary_key]:
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name]={}
                        
                    if "DW_AT_name" in tags:
                        
                        
                        for attr in DIE.attributes.values():
                            
                            if attr.name == "DW_AT_name": # VAR NAME 
                                PARAM_name = attr.value.decode("utf-8")
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}

                            if attr.name == "DW_AT_type":
                                var_type = DIE.tag.split('_')[-1]
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,attr.value) , 'kind':var_type}

                            # Check if this attribute contains location information
                            if loc_parser.attribute_has_location(attr, CU['version']):
                                loc = loc_parser.parse_from_attribute(attr,
                                                                      CU['version'])
    
#                                 print(CU_dictionary_key,FUNC_name,PARAM_name)
                                if isinstance(loc, LocationExpr):
#                                     print('      %s' % (
#                                         describe_DWARF_expr(loc.loc_expr,
#                                                             dwarfinfo.structs, CU.cu_offset)))
#                                     print(describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset))
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= describe_DWARF_expr(loc.loc_expr, dwarfinfo.structs, CU.cu_offset)
                                elif isinstance(loc, list):
#                                     print(show_loclist(loc,
#                                                        dwarfinfo,
#                                                        '      ', CU.cu_offset))
                                    FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name]["location"]= show_loclist(loc,
                                                       dwarfinfo,'      ', CU.cu_offset)
                        
                        

                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
                    

process_file(filePath)

Processing file: ./../../binaries/gnuit/src/gitfm
TEST!!!
  Found a compile unit at offset 0, length 10410
________________________________________
SUBPROGRAM:  write
SUBPROGRAM:  il_end
SUBPROGRAM:  panel_end
SUBPROGRAM:  il_echo
SUBPROGRAM:  panel_deactivate
SUBPROGRAM:  panel_activate
SUBPROGRAM:  panel_set_wrapped_isearch_flag
SUBPROGRAM:  strdup
SUBPROGRAM:  toprintable
SUBPROGRAM:  needs_quotes
SUBPROGRAM:  il_point
SUBPROGRAM:  il_history
SUBPROGRAM:  is_an_empty_command
SUBPROGRAM:  history_expand
SUBPROGRAM:  il_free
SUBPROGRAM:  panel_unselect_all
SUBPROGRAM:  tilde_expand
SUBPROGRAM:  tty_touch
SUBPROGRAM:  display_errors
SUBPROGRAM:  start
SUBPROGRAM:  is_a_bg_command
SUBPROGRAM:  il_message
SUBPROGRAM:  panel_set_focus
SUBPROGRAM:  get_local_time
SUBPROGRAM:  tty_goto
SUBPROGRAM:  alarm
SUBPROGRAM:  signal_handlers
SUBPROGRAM:  tty_get_screen
SUBPROGRAM:  panel_init
SUBPROGRAM:  status_init
SUBPROGRAM:  il_init
SUBPROGRAM:  title_init
SUBPROGRAM:  tty_start_cursorapp
SUBPR

In [6]:
# FUNC_PARAMS 


In [7]:
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment






In [8]:

fh = open(filePath, 'rb')
bin_bytearray = bytearray(fh.read())

In [9]:
# https://www.capstone-engine.org/lang_python.html



from capstone import *

from capstone.x86 import *


address_inst = {}
with open(filePath, 'rb') as f:
    elf = ELFFile(f)
    dwarfinfo = elf.get_dwarf_info()
    aranges = dwarfinfo.get_aranges()
    print(len(aranges.entries))
    for arange in aranges.entries:
        print(arange)
    for arange in aranges.entries:

        entry = arange.begin_addr
        exit  = arange.begin_addr + arange.length
        ops = bin_bytearray[entry: exit]

        md = Cs(CS_ARCH_X86, CS_MODE_64)
        md.detail = True
        for inst in md.disasm(ops, entry):

            address_inst[hex(inst.address)] = inst


            print('\n'*3)
            print(inst.address, inst.mnemonic+"  "+inst.op_str)
            (regs_read, regs_write) = inst.regs_access()

#             if len(inst.operands) > 0:
#                 c=-1
#                 for o in inst.operands:
#                     c += 1
#                     if o.type == CS_OP_MEM:
#                         print("\t\toperands[%u].type: MEM" %c)
#                         if o.value.mem.base != 0:
#                             print("\t\t\toperands[%u].mem.base: REG = %s" \
#                                 %(c, inst.reg_name(o.value.mem.base)))
#                         if o.value.mem.index != 0:
#                             print("\t\t\toperands[%u].mem.index: REG = %s" \
#                                 %(c, inst.reg_name(o.value.mem.index)))
#                         if o.value.mem.disp != 0:
#                             print("\t\t\toperands[%u].mem.disp: 0x%x" \
#                                 %(c, o.value.mem.disp))
#                         print(hex(o.value.mem.disp),o.value.mem.disp)



341
ARangeEntry(begin_addr=20192, length=5, info_offset=76698, unit_length=60, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=20441, length=72, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=20513, length=25, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=20538, length=813, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=21351, length=315, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=21666, length=298, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=21964, length=147, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=22111, length=94, info_offset=0, unit_length=460, version=2, address_size=8, segment_size=0)
ARangeEntry(begin_addr=22205, length=729, info_offset=0, unit_





56881 test  eax, eax




56883 je  0xdf03




56889 mov  rax, qword ptr [rip + 0x235d0]




56896 mov  rdx, qword ptr [rax + 0x48]




56900 mov  rax, qword ptr [rip + 0x235c5]




56907 sub  rdx, qword ptr [rbp - 0x18]




56911 mov  qword ptr [rax + 0x48], rdx




56915 mov  rax, qword ptr [rip + 0x235b6]




56922 mov  rdx, qword ptr [rax + 0x38]




56926 mov  rax, qword ptr [rip + 0x235ab]




56933 sub  rdx, qword ptr [rbp - 0x18]




56937 mov  qword ptr [rax + 0x38], rdx




56941 mov  rax, qword ptr [rip + 0x2359c]




56948 mov  rdx, qword ptr [rbp - 0x28]




56952 mov  qword ptr [rax + 0x18], rdx




56956 mov  rax, qword ptr [rip + 0x2358d]




56963 mov  rdx, qword ptr [rbp - 0x28]




56967 mov  qword ptr [rax + 0x20], rdx




56971 mov  rax, qword ptr [rip + 0x2357e]




56978 mov  rdx, qword ptr [rax + 0x58]




56982 mov  rax, qword ptr [rbp - 0x20]




56986 add  rdx, rax




56989 mov  rax, qword ptr [rip + 0x2356c]




56996 mov  rcx, qword ptr [rax + 0x58]









91367 mov  rdi, rax




91370 call  0x4960




91375 add  rax, 0x20




91379 mov  rdi, rax




91382 call  0x24650




91387 mov  qword ptr [rbp - 0x150], rax




91394 mov  rdx, qword ptr [rbp - 0x190]




91401 mov  rax, qword ptr [rbp - 0x150]




91408 lea  rcx, [rip + 0x123ea]




91415 mov  rsi, rcx




91418 mov  rdi, rax




91421 mov  eax, 0




91426 call  0x4dd0




91431 mov  rax, qword ptr [rbp - 0x150]




91438 mov  edx, 1




91443 mov  esi, 1




91448 mov  rdi, rax




91451 call  0x107d0




91456 mov  eax, 0




91461 call  0xb4b7




91466 mov  rax, qword ptr [rbp - 0x150]




91473 mov  rdi, rax




91476 call  0x1eeae




91481 lea  rdx, [rbp - 0xb0]




91488 mov  rax, qword ptr [rbp - 0x190]




91495 mov  rsi, rdx




91498 mov  rdi, rax




91501 call  0x1f0e4




91506 cmp  eax, -1




91509 jne  0x16581




91511 mov  eax, 6




91516 jmp  0x167c6




91521 lea  rdx, [rbp - 0x140]




91528 mov  rax, qword ptr [rbp - 0x198]




91535 mov  rsi, rdx










129602 add  ebx, 1




129605 movsxd  rax, ebx




129608 shl  rax, 3




129612 add  rax, r13




129615 mov  rax, qword ptr [rax]




129618 test  rax, rax




129621 jne  0x1f9d9




129623 add  r12d, 1




129627 cmp  r12d, r14d




129630 jl  0x1f9d2




129636 mov  eax, r14d




129639 add  rsp, 0x10




129643 pop  rbx




129644 pop  r12




129646 pop  r13




129648 pop  r14




129650 pop  rbp




129651 ret  




129652 endbr64  




129656 push  rbp




129657 mov  rbp, rsp




129660 push  r14




129662 push  r13




129664 push  r12




129666 push  rbx




129667 sub  rsp, 0x10




129671 mov  qword ptr [rbp - 0x28], rdi




129675 mov  r13, qword ptr [rip + 0x10d9e]




129682 mov  rax, qword ptr [rbp - 0x28]




129686 mov  rdi, rax




129689 call  0x4960




129694 mov  r14d, eax




129697 mov  ebx, 0




129702 jmp  0x1fb39




129707 movsxd  rdx, ebx




129710 mov  rax, qword ptr [rbp - 0x28]




129714 add  rax, rdx




129717 movzx  eax, byte ptr [rax]








154758 mov  r15, rdi




154761 push  r14




154763 mov  r14, rsi




154766 push  r13




154768 push  r12




154770 push  rbp




154771 push  rbx




154772 mov  rbx, rdx




154775 sub  rsp, 0x18




154779 call  0x4820




154784 test  rbx, rbx




154787 mov  rcx, r14




154790 mov  rdx, r15




154793 mov  r12d, dword ptr [rax]




154796 mov  rbp, rax




154799 lea  rax, [rip + 0xbfaa]




154806 cmove  rbx, rax




154810 xor  esi, esi




154812 xor  edi, edi




154814 mov  r8d, dword ptr [rbx]




154817 mov  r9, rbx




154820 call  0x25020




154825 mov  dword ptr [rbp], r12d




154829 lea  r13, [rax + 1]




154833 mov  dword ptr [rsp + 0xc], r12d




154838 mov  rdi, r13




154841 call  0x24650




154846 mov  r8d, dword ptr [rbx]




154849 mov  r9, rbx




154852 mov  rcx, r14




154855 mov  rdx, r15




154858 mov  rsi, r13




154861 mov  rdi, rax




154864 mov  r12, rax




154867 call  0x25020




154872 mov  eax, dword ptr [rsp + 0xc]




154876 mov 

In [10]:
from collections import defaultdict
import posixpath


In [11]:
addr_lineProgram ={}
addr_sourceFile = {}
with open(filePath, 'rb') as f:
    elffile = ELFFile(f)

    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        exit(0)

    dwarfinfo = elffile.get_dwarf_info()
    for CU in dwarfinfo.iter_CUs():
#         print(CU.get_top_DIE()['DW_AT_comp_dir'])
        CU_DIR_PATH = None
        CU_FILENAME = None
        for attr in CU.get_top_DIE().attributes.values():
            if attr.name == 'DW_AT_comp_dir':
                CU_DIR_PATH = attr.value.decode("utf-8")
            if attr.name == 'DW_AT_name':
                CU_FILENAME = attr.value.decode("utf-8")
            
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue

        for line_entry in line_program.get_entries():
            real_source_path = None
            if line_entry.state!=None:
                if line_entry.state.is_stmt!=0:

                    addr_lineProgram[hex(line_entry.state.address)] = line_entry

                    src_filename_from_lineentry = lpe_filename (line_program, line_entry.state.file)
                    if src_filename_from_lineentry == CU_FILENAME:
                        real_source_path = os.path.join(CU_DIR_PATH,CU_FILENAME)
                    else:
                        real_source_path = src_filename_from_lineentry
                    addr_sourceFile [hex(line_entry.state.address)] = real_source_path


        


  Found a compile unit at offset 0, length 10410
  Found a compile unit at offset 10414, length 7795
  Found a compile unit at offset 18213, length 931
  Found a compile unit at offset 19148, length 3815
  Found a compile unit at offset 22967, length 1824
  Found a compile unit at offset 24795, length 15615
  Found a compile unit at offset 40414, length 3379
  Found a compile unit at offset 43797, length 1585
  Found a compile unit at offset 45386, length 4279
  Found a compile unit at offset 49669, length 1102
  Found a compile unit at offset 50775, length 249
  Found a compile unit at offset 51028, length 1910
  Found a compile unit at offset 52942, length 710
  Found a compile unit at offset 53656, length 1720
  Found a compile unit at offset 55380, length 1383
  Found a compile unit at offset 56767, length 5736
  Found a compile unit at offset 62507, length 684
  Found a compile unit at offset 63195, length 4327
  Found a compile unit at offset 67526, length 1936
  Found a compile 

In [12]:
# print(addr_lineProgram)
# address_inst

def getSource(sourceFilePath, row , col):
    sourceFile = open(sourceFilePath, "r")
    fileContent = sourceFile.readlines()
    
    print(sourceFilePath, 'row: ',row , 'col ',col )
    row_content =  fileContent[row-1]
    
    
    if len(row_content)<col:
        return row_content
#     print(sourceFilePath, row , 'col ',col , 'len: ',len(row_content))
    print("*****",row_content,"))))")
    print('_',row_content[:(col-1)])
    print("_ _",row_content[col-1])
    print("_ _ _",row_content[col:])

    
    row_content = row_content[:(col-1)] + '|'+row_content[(col-1)]+'|' +row_content[col:]
    
    return row_content


In [13]:

# FUNCTION_DECL
# https://stackoverflow.com/questions/43460605/function-boundary-identification-using-libclang
# https://eli.thegreenplace.net/2011/07/03/parsing-c-in-python-with-clang


import clang.cindex



def get_all_var_types(source_path):
    srcFileName = source_path.split('/')[-1]

    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.VAR_DECL:
            # print(dir(f))
            print(f.extent.start.file.name)
            originFileName = f.extent.start.file.name.split('/')[-1]
            print(originFileName)
            print(f.displayname)
            print(f.type.spelling)
            print(f.extent.start.line,f.extent.start.column)
            print('\n')
            
            

   

def get_all_function_types(source_path):
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:
            # print(dir(f))
            print(f.displayname)
            print('function name: ',( f.spelling))
            print('Returns: ',(f.result_type.spelling))
            
            
            arg_len = len(list(f.type.argument_types()))
            if arg_len>0:
                arg_types = list(f.type.argument_types())
                for arg_type in arg_types:
                    print('arg_type:',arg_type.spelling)
                args = list(f.get_arguments())
                for arg in args:
                    print('arg:',arg.spelling)


            print("\n\n\n")

            
def get_function_boundaries(source_path):
    
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:

            function_name = f.displayname.split('(')[0]
            function_boundary_by_name[function_name]={}
            function_boundary_by_name[function_name] = { 'src_path':f.extent.start.file.name,
                              'src_file':f.extent.start.file.name.split('/')[-1],
                              'start_line':f.extent.start.line,
                              'start_col':f.extent.start.column,
                              'end_line':f.extent.end.line,
                              'end_col':f.extent.end.column}
    return function_boundary_by_name

def get_containing_function(source_file_path, line, col=0):
    function_boundary_by_name = get_function_boundaries(source_file_path)
    
    for function_name, item in function_boundary_by_name.items():
        if item['src_path'] == source_file_path:
            if line>= item['start_line'] and line<= item['end_line']:
                return function_name
        



In [14]:


with open('gitfm.s', 'w') as outFile:
    # outFile.write('file contents\n')
    lastSource = ""
    for address in address_inst:
        inst = address_inst[address]
        instrctionCode = (address+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)
        

        if int(address,0)< 0x24357:#0x23122:
            continue
        
        OFFSET = None
        if len(inst.operands) > 0 :
            c=-1
            for o in inst.operands:
                c += 1
                if o.type == CS_OP_MEM:
                    print("\t\toperands[%u].type: MEM" %c)
                    if o.value.mem.base != 0:
                        print("\t\t\toperands[%u].mem.base: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.base)))
                    if o.value.mem.index != 0:
                        print("\t\t\toperands[%u].mem.index: REG = %s" \
                            %(c, inst.reg_name(o.value.mem.index)))
                    if o.value.mem.disp != 0:
                        print("\t\t\toperands[%u].mem.disp: 0x%x" \
                            %(c, o.value.mem.disp))
                        OFFSET = o.value.mem.disp
                    print(hex(o.value.mem.disp),o.value.mem.disp)
                    
        
        if address in addr_lineProgram:
            
            srcFilePath =addr_sourceFile[address] 
#             srcFileName = srcFilePath.split('/')[-1]
            if srcFilePath!=lastSource:
                outFile.write("\n"+ '#'*100+"\n"+ srcFilePath.rjust(45) +'\n'+'#'*100+ "\n\n")
                lastSource = srcFilePath
            print("add",address)
            sourceCode = getSource(srcFilePath,addr_lineProgram[address].state.line, addr_lineProgram[address].state.column)
            function_name = get_containing_function(srcFilePath ,addr_lineProgram[address].state.line , addr_lineProgram[address].state.column)
#             print(function_name)
            
            if '\n' not in  sourceCode:
                sourceCode+=sourceCode+"\n"
            outFile.write(instrctionCode+"#"+ sourceCode  )
            print(instrctionCode+"#"+ sourceCode)
            
        else:
            
            outFile.write(instrctionCode+ '\n'  )
            print(instrctionCode)
        if OFFSET:
#             outFile.write("MEMORY OFFSET:     "+str(hex(OFFSET))+"     "+str(OFFSET)+'\n')
            pass

add 0x24357
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  86 col  7
*****       match = xmalloc (offsetof (struct userid, name) + strlen (name) + 1);
 ))))
_       
_ _ m
_ _ _ atch = xmalloc (offsetof (struct userid, name) + strlen (name) + 1);

0x24357:	call 0x24650                        #      |m|atch = xmalloc (offsetof (struct userid, name) + strlen (name) + 1);

0x2435c:	mov rsi, r12                        
		operands[0].type: MEM
			operands[0].mem.base: REG = rax
0x0 0
0x2435f:	mov dword ptr [rax], ebp            
		operands[1].type: MEM
			operands[1].mem.base: REG = rax
			operands[1].mem.disp: 0x10
0x10 16
0x24361:	lea rdi, [rax + 0x10]               
0x24365:	mov rbx, rax                        
add 0x24368
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  79 col  3
***** 	}
 ))))
_ 	}
_ _ 

_ _ _ 
0x24368:	call 0x4870                         #	}|
|
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xd7dc
0xd7dc 55260
add 0x2

0x2444e:	call 0x4870                         #	}|
|
add 0x24453
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  134 col  3
*****   if (pwent)
 ))))
_   
_ _ i
_ _ _ f (pwent)

0x24453:	test r12, r12                       #  |i|f (pwent)

0x24456:	je 0x24480                          
		operands[1].type: MEM
			operands[1].mem.base: REG = r12
			operands[1].mem.disp: 0x10
0x10 16
add 0x24458
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  136 col  7
*****       tail->id.u = pwent->pw_uid;
 ))))
_       
_ _ t
_ _ _ ail->id.u = pwent->pw_uid;

0x24458:	mov eax, dword ptr [r12 + 0x10]     #      |t|ail->id.u = pwent->pw_uid;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbx
0x0 0
0x2445d:	mov dword ptr [rbx], eax            
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xd6ea
0xd6ea 55018
add 0x2445f
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  137 col  7
*****       tail->next = user_alist;
 ))))
_       
_ _ t
_ _ _

0x24518:	call 0x4870                         #	}|
|
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xd61c
0xd61c 54812
add 0x2451d
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  173 col  7
*****       match->next = group_alist;
 ))))
_       
_ _ m
_ _ _ atch->next = group_alist;

0x2451d:	mov rax, qword ptr [rip + 0xd61c]   #      |m|atch->next = group_alist;

		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].mem.disp: 0xd615
0xd615 54805
0x24524:	mov qword ptr [rip + 0xd615], rbx   
		operands[0].type: MEM
			operands[0].mem.base: REG = rbx
			operands[0].mem.disp: 0x8
0x8 8
0x2452b:	mov qword ptr [rbx + 8], rax        
add 0x2452f
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  174 col  7
*****       group_alist = match;
 ))))
_       
_ _ g
_ _ _ roup_alist = match;

0x2452f:	jmp 0x244cd                         #      |g|roup_alist = match;

		operands[0].type: MEM
			operands[0].mem.base: REG = rax
			oper

0x24603:	test r12, r12                       #  |i|f (grent)

0x24606:	je 0x24630                          
		operands[1].type: MEM
			operands[1].mem.base: REG = r12
			operands[1].mem.disp: 0x10
0x10 16
add 0x24608
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  218 col  7
*****       tail->id.g = grent->gr_gid;
 ))))
_       
_ _ t
_ _ _ ail->id.g = grent->gr_gid;

0x24608:	mov eax, dword ptr [r12 + 0x10]     #      |t|ail->id.g = grent->gr_gid;

		operands[0].type: MEM
			operands[0].mem.base: REG = rbx
0x0 0
0x2460d:	mov dword ptr [rbx], eax            
		operands[1].type: MEM
			operands[1].mem.base: REG = rip
			operands[1].mem.disp: 0xd52a
0xd52a 54570
add 0x2460f
/home/nahid/reverse/binaries/gnuit/lib/idcache.c row:  219 col  7
*****       tail->next = group_alist;
 ))))
_       
_ _ t
_ _ _ ail->next = group_alist;

0x2460f:	mov rax, qword ptr [rip + 0xd52a]   #      |t|ail->next = group_alist;

		operands[0].type: MEM
			operands[0].mem.base: REG = rip
			operands[0].

IndexError: list index out of range

In [ ]:
0x24698